<a href="https://colab.research.google.com/github/Moad-S/Dog-Specie-Detector/blob/main/annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



import libraries



In [ ]:
import glob
import os
import pickle
import random
import shutil
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join


mounting google drive folder into collab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

extracting the standford dataset into collab


In [ ]:
!mkdir dataset
!unzip /content/drive/MyDrive/dogdata.zip -d /content/dataset

Adding the annotation directory and all sub directories and files into the variable "dirs"

In [ ]:
dirs = glob.glob('/content/dataset/annotations/Annotation' + '/*')
classes = []

function that returns a list of all files in a folder

In [ ]:
def getFilesInDir(dir_path):
    file_list = []
    for filename in glob.glob(dir_path + '/*'):
        if os.path.isdir(filename):
          continue
        file_list.append(filename)
    return file_list

function that takes the size (width, height, depth) and the bounding box coordinates (Xmin, Ymin, Xmax, Ymax)

In [ ]:
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

A wrapper function that converts an annotation file from xlm format to Yolo format.

annotation_path: full path of the annotation file

output_path: path of the ouput folder


In [ ]:
def convert_annotation(annotation_path,output_path):
    basename = os.path.basename(annotation_path)
    basename_no_ext = os.path.splitext(basename)[0]

    in_file = open(annotation_path )
    out_file = open(output_path + '/'+ basename_no_ext + '.txt', 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        cls = obj.find('name').text
        if cls not in classes:
            classes.append(cls)
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        #print(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

loop that iterates through all directories of the annotation folder and convert them to the Yolo format

In [ ]:
for dir_path in dirs:
    annotation_paths = getFilesInDir(dir_path)
    output_path = dir_path +'/yolo/'
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    for image_path in annotation_paths:
        convert_annotation(image_path, output_path)
    print(dir_path)

In [ ]:
print(classes)

In [ ]:
print("Finished processing: " + dir_path)

create the folder structure for yolo 

In [ ]:
!mkdir -p data/images/train data/images/test data/images/val  data/annotations/train data/annotations/test data/annotations/val

In [ ]:
dirs = glob.glob('/content/dataset/annotations/Annotation' + '/*')




def getFilesInDir(dir_path):
    file_list = []
    for filename in glob.glob(dir_path + '/*'):
        if os.path.isdir(filename):
          continue
        file_list.append(filename)
    return file_list

function that finds the corresponding image of an annotation file 

In [ ]:
def positionData(annotation_file_list,output_path,output_type,images_path):
  image_list=glob.glob(images_path + '/*/*.jpg')  
  for annotation_file in annotation_file_list:
    base_name=os.path.basename(annotation_file)
    file_name=os.path.splitext(base_name)[0]
    cls_id = [i for i, item in enumerate(image_list) if item.endswith(file_name+'.jpg')][0]
    image_file=image_list[cls_id]
    shutil.copy2(image_file,output_path+'/images/'+output_type)
    shutil.copy2(annotation_file,output_path+'/annotations/'+output_type)

Shuffling Yolo annotations and splitting the data to train, test and validation data, as well as putting the corresponsding images in their respective folders

In [ ]:
trainCoeef=0.8
testCoeef=0.1
valCoeef=0.1
for dir_path in dirs:
  yolo_annotation_paths = getFilesInDir(dir_path+'/yolo/')
  random.shuffle(yolo_annotation_paths)

  length = len(yolo_annotation_paths)

  trainIndex = round(length*trainCoeef)
  testIndex = trainIndex+round(length*testCoeef)

  train_files=yolo_annotation_paths[0:trainIndex]
  test_files=yolo_annotation_paths[trainIndex:testIndex]
  val_files=yolo_annotation_paths[testIndex:]
  dataOutputPath='/content/data'
  imagesPath='/content/dataset/images/Images'
  positionData(train_files,dataOutputPath,'train',imagesPath)
  positionData(test_files,dataOutputPath,'test',imagesPath)
  positionData(val_files,dataOutputPath,'val',imagesPath)
